<h2> MBA Data Workpace

In [1]:
import os
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/andreasfreund/.config/gcloud/application_default_credentials.json"
%load_ext google.cloud.bigquery

pd.set_option('display.max_rows', 150)



<h3> Python dictionaries for ISP tiers used in MBA report and used in sample data

In [ ]:
centurylink_tiers = {"tech": "DSL", 
                       "download_tiers": [1.5, 3, 10, 12, 20, 25, 40, 80, 100], 
                       "up_tiers": [.512, .896, 2, 5, 10]
                      }
# print(centurylink_tiers)
print(centurylink_tiers["download_tiers"])

Query for min and max speeds for whitebox matching table by ISP and tech

In [ ]:
%%bigquery

SELECT COUNT(*) as total, MIN(mt.Download) as min_download, MAX(mt.Download) as max_download,
        MIN(mt.Upload) as min_upload, MAX(mt.Upload) as max_Upload, mt.ISP, mt.Technology  
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt
GROUP BY mt.ISP, mt.Technology

query for finding total number of whiteboxes per ISP and Technology

In [6]:
%%bigquery


SELECT COUNT(mt.Unit_ID) as isp_box_count, mt.ISP, mt.Technology
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt
GROUP BY mt.ISP, mt.Technology

Downloading: 100%|████████████████████████████| 16/16 [00:01<00:00, 12.03rows/s]


,isp_box_count,ISP,Technology
0,212,Cox,Cable
1,319,Charter,Cable
2,453,Comcast,Cable
3,160,Optimum,Cable
4,288,Verizon,Fiber
5,225,Frontier,DSL
6,276,Frontier,Fiber
7,187,Mediacom,Cable
8,322,Windstream,DSL
9,705,CenturyLink,DSL


query for finding distinct download speeds by whitebox, ISP, and Technology

In [ ]:
%%bigquery

SELECT DISTINCT mt.Download, mt.ISP, mt.Technology
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt



query for finding distinct upload speeds by whitebox, ISP, and Technology

In [ ]:
%%bigquery

SELECT DISTINCT mt.Upload, mt.ISP, mt.Technology
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt

sample query for inner-join with dns test data and unit profile

In [ ]:
%%bigquery

SELECT dns_test.unit_id, dns_test.ddate, dns_test.rtt, mt.Unit_ID, mt.ISP, mt.Technology, mt.Census
FROM `broadband-data.mba_data.curr_dns` as dns_test
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON dns_test.unit_id=mt.Unit_ID
ORDER BY mt.ISP
LIMIT 100

In [ ]:
bytes_sec = 14447759

mbps = bytes_sec/125000

print(mbps)

Mean and approx median RTT for all ISP download speed tiers


*This uses the approximate median formula for a simpler query but is not an exact calculation*

In [ ]:
%%bigquery


# SELECT PERCENTILE_CONT(dns.rtt, .5) OVER() As median_rtt
# SELECT avg(dns.rtt) As avg_rtt
# SELECT PERCENTILE_CONT(dns.rtt, .5) OVER() As median_rtt, mt.ISP, mt.Download
# SELECT COUNT(dns.unit_id) as dns_tests_per_service_tier, mt.Download as service_tier
# SELECT PERCENTILE_CONT(dns.rtt, .5) OVER() as median_rtt, mt.Download as service_tier
# SELECT COUNT(dns.rtt) as dns_tests_per_service_tier, mt.Download as service_tier, avg(dns.rtt) As avg_rtt_per_service_tier




SELECT 
    mt.Download as service_tier, 
    APPROX_QUANTILES(dns.rtt, 100)[OFFSET(50)] as approx_median_rtt, 
    avg(dns.rtt) as avg_rtt_per_service_tier, mt.ISP as ISP
FROM `broadband-data.mba_data.curr_dns` as dns
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON dns.unit_id=mt.Unit_ID
# WHERE mt.ISP = "Comcast"
GROUP BY service_tier, ISP
ORDER BY service_tier
Limit 1000






Sample query for exact median rtt for comcast at specified download tier 

In [5]:
%%bigquery


SELECT 
    PERCENTILE_CONT(httpmt.bytes_sec/125000, .5) OVER() as median_peak_download,
    PERCENTILE_CONT(httpmt.bytes_sec/125000, .3) OVER() as thirty_pct_download,
    mt.Download as service_tier
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID 
# WHERE mt.ISP = "Comcast" and mt.Download = 25
# LIMIT 
group by service_tier

Executing query with job ID: 4a0a5307-18b6-4b73-a318-0c38fd0a06bc
Query executing: 0.34s


ERROR:
 400 SELECT list expression references httpmt.bytes_sec which is neither grouped nor aggregated at [2:21]

Location: US
Job ID: 4a0a5307-18b6-4b73-a318-0c38fd0a06bc



Unique whiteboxes per service-tier for download speed tests, using IPv4 and multiple concurrent TCP connections

In [7]:
%%bigquery

SELECT 
    COUNT(DISTINCT httpmt.unit_id) as unique_boxes_tested_per_service_tier, 
    mt.Download as service_tier, 
    mt.ISP as ISP
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
GROUP BY service_tier, ISP
ORDER BY service_tier
LIMIT 1000


Downloading: 100%|██████████████████████████| 124/124 [00:01<00:00, 88.66rows/s]


,unique_boxes_tested_per_service_tier,service_tier,ISP
0,1,0.768,CenturyLink
1,7,1.000,Verizon DSL
2,3,1.000,Frontier
3,65,1.500,CenturyLink
4,50,3.000,CenturyLink
5,15,3.000,Windstream
6,10,3.000,Comcast
7,24,3.000,Frontier
8,23,3.000,Verizon DSL
9,2,4.000,CenturyLink


Total download speed tests per service-tier, using IPv4 and multiple concurrent TCP connections

In [ ]:
%%bigquery

SELECT 
    mt.Download as service_tier, 
    count(httpmt.unit_id) as total_tests_per_tier, 
    mt.ISP as ISP
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
GROUP BY service_tier, ISP
ORDER BY service_tier
LIMIT 1000

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

In [151]:
#panda test



sql = """
SELECT
    mt.Download as service_tier, 
    count(httpmt.unit_id) as total_tests_per_tier, 
    mt.ISP as ISP
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
GROUP BY service_tier, ISP
ORDER BY service_tier
LIMIT 1000
"""

sql2 = """
SELECT 
    mt.Download as download_service_tier,
    mt.ISP as ISP,
    mt.Technology as tech,
    avg(httpmt.bytes_sec/125000) as avg_Mbps,
    APPROX_QUANTILES(httpmt.bytes_sec/125000, 100)[OFFSET(50)] as approx_median_Mbps,
    MAX(httpmt.bytes_sec/125000) as max_Mbps, 
    MIN(httpmt.bytes_sec/125000) as min_Mbps
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
WHERE (EXTRACT(HOUR FROM httpmt.dtime) between 19 AND 23) AND ISP = "Comcast"
GROUP BY download_service_tier, ISP, tech
ORDER BY download_service_tier
"""

df = client.query(sql).to_dataframe()
df2 = client.query(sql2).to_dataframe()
df.head()


,service_tier,total_tests_per_tier,ISP
0,0.768,703,CenturyLink
1,1.000,1712,Verizon DSL
2,1.000,874,Frontier
3,1.500,23952,CenturyLink
4,3.000,12676,CenturyLink


In [152]:
df2

,download_service_tier,ISP,tech,avg_Mbps,approx_median_Mbps,max_Mbps,min_Mbps
0,3.0,Comcast,Cable,3.511462,3.556472,3.607728,0.262824
1,15.0,Comcast,Cable,16.797835,17.828472,17.884024,9.386312
2,25.0,Comcast,Cable,28.419018,29.699632,30.241992,0.197568
3,35.0,Comcast,Cable,41.647737,41.646128,41.935408,41.442992
4,70.0,Comcast,Cable,81.138179,82.616488,83.593912,64.365872
5,75.0,Comcast,Cable,86.883362,88.877400,89.875416,26.249472
6,100.0,Comcast,Cable,114.927412,118.729664,121.017336,0.227976
7,105.0,Comcast,Cable,112.864438,114.152488,120.711112,92.254640
8,175.0,Comcast,Cable,201.187753,207.945368,208.998904,12.501008
9,200.0,Comcast,Cable,229.356936,235.659080,350.955728,0.129736


<h4>Approx median and average download speed per service_tier and ISP in Mbps during peak hours<h4>

In [ ]:
%%bigquery

SELECT 
    mt.Download as download_service_tier,
    mt.ISP as ISP,
    mt.Technology as tech,
    avg(httpmt.bytes_sec/125000) as avg_Mbps,
    APPROX_QUANTILES(httpmt.bytes_sec/125000, 100)[OFFSET(50)] as approx_median_Mbps,
    MAX(httpmt.bytes_sec/125000) as max_Mbps, 
    MIN(httpmt.bytes_sec/125000) as min_Mbps
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
# WHERE EXTRACT(HOUR FROM httpmt.dtime) between 19 AND 23
WHERE (EXTRACT(HOUR FROM httpmt.dtime) between 19 AND 23) AND ISP = "Comcast"
GROUP BY download_service_tier, ISP, tech
ORDER BY download_service_tier

<h4>Approx median and average download speed per service_tier and ISP in Mbps during off-peak hours<h4>

In [ ]:
%%bigquery

SELECT 
    mt.Download as download_service_tier,
    mt.ISP as ISP,
    mt.Technology as tech,
    avg(httpmt.bytes_sec/125000) as avg_Mbps,
    APPROX_QUANTILES(httpmt.bytes_sec/125000, 100)[OFFSET(50)] as approx_median_Mbps,
    MAX(httpmt.bytes_sec/125000) as max_Mbps, 
    MIN(httpmt.bytes_sec/125000) as min_Mbps
FROM `broadband-data.mba_data.curr_httpgetmt` as httpmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httpmt.unit_id=mt.Unit_ID
# WHERE EXTRACT(HOUR FROM httpmt.dtime) NOT between 19 AND 23
WHERE (EXTRACT(HOUR FROM httpmt.dtime) NOT between 19 AND 23) AND ISP = "Comcast"
GROUP BY download_service_tier, ISP, tech
ORDER BY download_service_tier

<h4>approx median and average upload speed per service_tier and ISP in Mbps<h4>

In [ ]:
%%bigquery

SELECT
    mt.Upload as upload_service_tier,
    mt.ISP as ISP,
    mt.Technology as tech,
    avg(httppmt.bytes_sec/125000) as avg_Mbps,
    APPROX_QUANTILES(httppmt.bytes_sec/125000, 100)[OFFSET(50)] as approx_median_Mbps,
    MAX(httppmt.bytes_sec/125000) as max_Mbps, 
    MIN(httppmt.bytes_sec/125000) as min_Mbps
FROM `broadband-data.mba_data.curr_httppostmt` as httppmt
INNER JOIN `broadband-data.mba_data.unit-profile-sept2020` as mt ON httppmt.unit_id=mt.Unit_ID
# WHERE ISP = "Comcast"
GROUP BY upload_service_tier, ISP, tech
ORDER BY upload_service_tier


median download for all whiteboxes

In [ ]:
%%bigquery

SELECT PERCENTILE_CONT(mt.Download, .5) OVER () As median
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt
LIMIT 1

average download for all whiteboxes

In [ ]:
%%bigquery

SELECT avg(mt.Download) as avg_download
FROM `broadband-data.mba_data.unit-profile-sept2020` as mt
Limit 1